# VIME (Value Imputation and Mask Estimation)

> "논문 리뷰"

- toc: true
- branch: master
- badges: true
- comments: true
- hide: false
- author: 한재수
- categories: [Paper Review]

`-` 논문 링크: https://vanderschaar-lab.com/papers/NeurIPS2020_VIME.pdf

`-` 논문 구현: https://github.com/Jaesu26/vime

`-` 핵심 아이디어 알아 보기

## 개요

`-` 이미지넷과 같이 라벨링된 많은 데이터를 기반으로 많은 지도 학습 모델이 발전했다

`-` 하지만 현실에서 라벨링된 데이터를 많이 수집하는 것은 비싸고 때로는 불가능하다

`-` 라벨링만 되지 않았을 뿐 데이터는 많이 존재하는데 이를 활용한 것이 자기 지도 학습과 준 지도 학습이다

`-` 하지만 현존하는 자기 지도 학습과 준 지도 학습은 정형 데이터에 효과적이지 않다

`-` 자기 지도 학습과 준 지도 학습은 라벨링 되지 않은 데이터를 어떻게 활용하느냐가 중요하다

`-` 원본 데이터에 약간의 변형을 가한 후 복원한 정보를 사용해 원본과 복원된 것의 차이를 학습한다 (pretext task)

`-` 이미지는 회전, 색상 변환 등 변형 방식이 다양하지만 정형 데이터는 그렇지 않다 (정형 데이터에 회전 변환?)

`-` VIME은 mask vector estimation과 feature vector estimation이라는 새로운 pretext tasks를 제안하여 자기 지도 학습을 해결하고자 한다

`-` 또한, 새로운 정형 데이터 augmentation method을 제안한다

## Self-supervised learning (VIME)

![](my_icons/vime-self-diagram.png)

`-` 새로운 2가지 pretext tasks: feature vector esitimation, mask vector estimation

`-` 2가지 pretext tasks는 하나의 pretext 분포 $P_{X_s, Y_s}$를 공유한다

`-` 첫 번째로 mask vector 제너레이터는 $\mathbf{m} = [m_1, \dots, m_d]^\top \in \{0, 1\}^d$를 생성한다

`-` $m_j$는 마스킹할 확률이 $p_m$인 베르누이 분포에서 샘플링된 것이다 (i.e. $p_{\mathbf{m}} = \prod\limits_{j=1}^{d}\operatorname{Bern}\left(m_j|p_m\right))$

`-` 두 번째로 pretext 제너레이터 $g_m: \mathcal{X} \times \{0, 1\}^d \to \mathcal{X}$은 라벨링되지 않은 $\mathcal{D}_u$에서 샘플링한 $\mathbf{x}$와 $\textbf{m}$을 사용해 $\mathbf{\tilde x}$를 생성한다

`-` 이때 $\mathbf{\tilde x} = g_m\left(\mathbf{x}, \mathbf{m}\right) =  \mathbf{m} \odot \mathbf{\bar x} + (1-\mathbf{m}) \odot \mathbf{x}$

`-` $\mathbf{\bar x}$는 $\mathbf x$의 각 피쳐에 대해 랜덤 셔플을 적용한 행렬이다

`-` 즉, 마스킹 안된 곳은 원본을 사용하고 마스킹된 곳은 랜덤 셔플한 것을 사용하겠다는 의미이다

`-` 인코더 $e$는 $\mathbf{\tilde x}$를 $\mathbf z$로 재표현한 후 pretext predictive model이 $ \mathbf z$로부터 원래의 $\mathbf x$를 추정한다

`-` 위의 문제를 해결하기 위해 2개의 서브 태스크로 나누어 해결한다

`1.` mask vector estimation: 어떤 피쳐가 마스크 되었는지 예측한다

`-` $s_m: \mathcal{Z} \to \{0, 1\}^d$, $\mathbf z$를 입력으로 사용해 $\mathbf{\hat{m}}$을 추정한다

`2.` feature vector esimation: 마스킹된 피쳐의 원래 값을 추정한다

`-` $s_r: \mathcal{Z} \to \mathcal{X}$, $\mathbf z$를 입력으로 사용해 $\mathbf{\hat{x}}$을 추정한다

`-` $e$와 $s_m, s_r$은 다음의 최적화 문제에서 함께 학습된다

$$\min_{e,s_m,s_r} \mathbb{E}_{\mathbf{x} \sim P_X, \mathbf{m} \sim P_{\mathbf m},\mathbf{\bar x} 
\sim g_m(\mathbf{x},\mathbf{m})}\left[l_m\left(\mathbf{m},\mathbf{\hat m}\right) + \alpha \cdot l_r\left(\mathbf{x}, \mathbf{\hat x}\right)\right]$$

`-` 이때 $\mathbf{\hat m} = (s_m \circ e)(\mathbf{\tilde x}),\quad \mathbf{\hat x} = (s_r \circ e )(\mathbf{\tilde x})$

$$l_m\left(\mathbf{m}, \mathbf{\hat m}\right) = -\frac{1}{d}\left[\sum\limits_{j=1}^{d}m_j\log\left[(s_m\circ e)_j(\mathbf{\tilde x})\right]
+ (1- m_j)\log\left[(1- s_m\circ e)_j(\mathbf{\tilde x})\right]\right]$$

`-` $l_m$은 마스크 벡터 로스로 마스크 벡터의 각 차원에 대해 바이너리 크로스 엔트로피를 계산한 후 평균낸 것이다

`-` $l_r$은 재구성 로스 (reconstruction loss)로 다음과 같다

$$l_r\left(\mathbf x,\mathbf{\hat x}\right) = \frac{1}{d}\left[\sum\limits_{j=1}^{d}\left(x_j - (s_r \circ e)_j(\mathbf{\tilde x})\right)^2\right]$$

`-` 범주형 변수의 경우 MSE loss 대신에 cross entropy loss를 사용한다

`-` 이 과정에서 인코더 $e$는 $\mathbf{x}$에서 상관관계가 높은 변수들을 포착하는 법과 $\mathbf{x}$로 복구할 수 있는 잠재 공간 $\mathbf{z}$를 만드는 법을 배운다

## Semi-supervised learning (VIME)

![](my_icons/vime-semi-diagram.png)

`-` 여기서는 self-supervised learning에서 학습된 인코더 $e$를 semi-supervised learning에 활용하는 법을 보인다

`-` $f_e = f \circ e,\; \hat{y}=f_e(\mathbf{x})$, predictive model $f$를 다음의 목적 함수를 최소화하는 방향으로 학습시킨다

$$\mathcal{L}_{\text{final}} = \mathcal{L}_s + \beta \cdot \mathcal{L}_u$$

`-` supervised loss $\mathcal{L}_s$는 다음과 같다

$$ \mathcal{L}_s = \mathbb{E}_{(\mathbf{x},y)\sim P_{X,Y}}\left[l_s\left(y, f_e(\mathbf{x})\right)\right]$$

`-` $l_s$는 일반적인 지도 학습에서 사용되는 손실 함수로 MSE loss나 cross entropy loss같은 것이다

`-` unsupervised(consistency) loss $\mathcal{L}_u$는 원래 샘플 $\mathbf{x}$와 재구성된 $\mathbf{\tilde x}$로 정의된다

$$\mathcal{L}_u = \mathbb{E}_{\mathbf{x}\sim P_X,\mathbf{m}\sim p_{\mathbf{m}}, \mathbf{\tilde x}\sim g_m(\mathbf{x},\mathbf{m})} \left[\left(f_e(\mathbf{\tilde x}) - f_e(\mathbf{x})\right)^2\right]$$

`-` consistency loss는 모델 $f$가 $\mathbf{x}$에 약간의 변형을 가하더라도 기존과 동일한 $y$로 예측하도록 만든다

`-` 약간의 변형이라면 원본 클래스인 $y$가 손상되지 않을 것이므로 합리적인 생각이며 이를 통해 라벨링되지 않은 데이터를 활용할 수 있게 만든다

`-` consistency loss ($\mathcal{L}_u$)에 집중해보자

`-` 고정된 샘플 $\mathbf{x}$에 대해 $\mathcal{L}_u$는 $p_{\mathbf{m}}$과 $g_m(\mathbf{x},\mathbf{m})$에 관한 식이며 $f_e(\mathbf{\tilde{x}})$의 분산으로 해석될 수 있다

`-` 약간의 변형이 가해져도 동일한 클래스여야 하므로 합리적인 방식이다

`-` $\mathcal{L}_u$를 $f_e(\mathbf{\tilde{x}})$의 분산으로 해석한다면 $\mathcal{L}_u$의 추정량 $\hat{\mathcal{L}}_u$는 다음과 같다

$$\hat{\mathcal{L}}_u = \frac{1}{N_b K}\sum\limits_{i=1}^{N_b}\sum\limits_{k=1}^{K}
\left[\left(f_e(\mathbf{\tilde{x}}_{i,k}) - f_e(\mathbf{x}_i)\right)^2\right]
=\frac{1}{N_b K}\sum\limits_{i=1}^{N_b}\sum\limits_{k=1}^{K}
\left[\left(f(\mathbf{z}_{i,k}) - f(\mathbf{z}_i)\right)^2\right]$$

`-` 에폭마다 배치로부터 $K$개의 증강 샘플 $\mathbf{\tilde x}_1,\dots,\mathbf{\tilde x}_K$이 생성된다

`-` $N_b$는 배치 크기, $\mathbf{z}_i$는 $f_e(\mathbf{x})$이며 $\mathbf{z}_{i,k},\,k=1,\dots,K$는  $\mathbf{\tilde x}_1,\dots,\mathbf{\tilde x}_K$로부터 만든 것이다

`-` 훈련이 끝난 후 테스트 샘플의 아웃풋은 $\hat{y} = f_e\left(\mathbf{x}^t\right)$와 같이 얻어진다

## 부록

`-` 분산: $Var(X) = E\left(X-E(X)\right)^2$